In [56]:
pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pydub import AudioSegment as audio
import os

In [3]:
unproccessed_data_dir = os.path.abspath('') + "/Unproccessed_Wav_Files"

Steps
- go through all files in the unprocessed wav files
- break up all files so they are split by 1 second
- use regex to get the file name minus the .wav at the end to make new 1 second wav files
- save each of the files into the processed wav folder

In [7]:
for filename in os.listdir(unproccessed_data_dir):
    if filename.endswith(".wav"):
        hearing = audio.from_wav(os.path.join(unproccessed_data_dir,filename))
        speaker = filename.removesuffix(".wav")
        t1 = 0
        t2 = 1000
        second = 1
        total_time = hearing.duration_seconds*1000
        while t2 <= total_time:
            new_audio = hearing[t1:t2]
            new_audio.export(f"{os.path.abspath('')}/Processed_Wav_Files/{speaker}_{second}.wav", format = "wav");
            t1 += 1000
            t2 += 1000
            second += 1